<p style="font-family:Times New Roman, serif; font-size:19px;text-align:justify">
Acá buscaré retornos y alphas basados en 10 señales cuantitativas comunes con el fin de ver una concordancia con el retorno esperado encontrado en el proyecto anterior, además de ver si uno de estos alpha es concordante con el exceso de retorno obtenido al usar el promedio de los 5 años. Las 10 condiciones a usar son: 
<ol>
  <li><b>Mercado Alcista:</b> Precio Mercado &gt; MM200 Mercado</li>
  <li><b>Fuerza Relativa:</b> MA200 Activo &gt; MM200 Mercado</li>
  <li><b>Sobreventa (RSI):</b> RSI_14 Activo &lt; 30</li>
  <li><b>Sobrecompra (RSI):</b> RSI_14 Activo &gt; 70</li>
  <li><b>Pánico (VIX):</b> VIX &gt; 25</li>
  <li><b>Calma (VIX):</b> VIX &lt; 15</li>
  <li><b>Evento PEAD:</b> 20 días post-reporte de ganancias</li>
  <li><b>Evento FOMC:</b> Días de anuncio de la Fed</li>
  <li><b>Días Extremos (Activo):</b> Retorno Activo &gt; 3-Sigma</li>
  <li><b>Filtro de Calidad (Mercado):</b> Excluir días donde Retorno Mercado &gt; 2-Sigma</li>
</ol>
</p>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

<p style="font-family:Times New Roman, serif; font-size:19px;">
Primero se definen las funciones para poder calcular la desviación estándar y el Rsi.
</p>

In [2]:
def calcular_rsi(precios, ventana=14):

    delta = precios.diff(1)

    ganancias = delta.clip(lower=0)
    perdidas = -delta.clip(upper=0)
    # Se usa ewm (media móvil exponencial) que es el estándar para RSI
    avg_ganancias = ganancias.ewm(com=ventana - 1, min_periods=ventana).mean()
    avg_perdidas = perdidas.ewm(com=ventana - 1, min_periods=ventana).mean()
    
    #Fuerza Relativa (RS)
    rs = avg_ganancias / avg_perdidas
    
    # RSI
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calcular_sigma_movil(retornos, ventana=20):
    "(Desviación estándar móvil de N días)"
    return retornos.rolling(window=ventana).std()

In [3]:
tickers_cartera = ['GOOGL', 'JNJ', 'KO'] 
weights = np.array([0.4, 0.3, 0.3]) 
ticker_mercado = 'SPY'
ticker_vix = '^VIX'
start_date = '2020-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d') # Usa 'hoy' como fecha final
RF_ANUAL = 0.03
ERM_ANUAL = 0.08

In [4]:
def preparar_datos(tickers_cartera, weights, ticker_mercado, ticker_vix, start, end):

    todos_tickers = tickers_cartera + [ticker_mercado, ticker_vix]
    try:
        precios = yf.download(todos_tickers, start=start, end=end)['Close']
    except Exception as e:
        print(f"Error al descargar datos: {e}")
        return None

    precios = precios.rename(columns={ticker_vix: 'VIX'})
    retornos = np.log(precios / precios.shift(1))
    
    # Calcular el retorno de la cartera
    # Aseguramos que solo usamos tickers de la cartera para el .dot()
    retornos_activos_df = retornos[tickers_cartera]
    retornos['Cartera'] = retornos_activos_df.dot(weights)
    base_index = 100
    cumulative_log_returns = retornos['Cartera'].cumsum()
    precios_cartera = base_index * np.exp(cumulative_log_returns)

    # 4. Ahora calculamos el RSI sobre esta serie de precios
    retornos['RSI_Cartera'] = calcular_rsi(precios_cartera)
    
    # 5. Añadir precios e indicadores al DataFrame de retornos para filtrar
    retornos[f'Precio_{ticker_mercado}'] = precios[ticker_mercado]
    retornos['MM50'] = precios[ticker_mercado].rolling(50).mean()
    retornos['MM200'] = precios[ticker_mercado].rolling(200).mean()
    retornos[f'Vol_20d_{ticker_mercado}'] = calcular_sigma_movil(retornos[ticker_mercado], 20)
    retornos['Cond_Dias_Normales'] = retornos[f'Precio_{ticker_mercado}'].abs() < (2 * retornos[f'Vol_20d_{ticker_mercado}'])
    

    for ticker in tickers_cartera:
        retornos[f'Precio_{ticker}'] = precios[ticker]
        retornos[f'MA50_{ticker}'] = precios[ticker].rolling(50).mean()
        retornos[f'MA200_{ticker}'] = precios[ticker].rolling(200).mean()
        retornos[f'RSI_{ticker}'] = calcular_rsi(precios[ticker])
        retornos[f'Vol_20d_{ticker}'] = calcular_sigma_movil(retornos[ticker], 20)
    
    retornos['VIX'] = precios['VIX']
    retornos = retornos.rename(columns={ticker_mercado: 'Mercado'})
    datos_completos = retornos.dropna()
    
    print(f"Datos preparados. {len(datos_completos)} días hábiles listos para analizar.")
    return datos_completos

<p style="font-family:Times New Roman, serif; font-size:19px;text-align:justify">
Luego de definir todos los datos que necesitamos obtener para usar las condiciones, se realiza una función que reciba los datos filtrados a través de la función query, y así realizar el modelo CAPM para los días que se cumplan, siempre que la cantidad de días sea mayor a 30 para que se puedan considerar significativos.
</p>

In [5]:
def probar_condicion(datos_completos, target_analisis, filtro_condicion, rf, erm):
   
    try:
        datos_filtrados = datos_completos.query(filtro_condicion)
    except Exception as e:
        # print(f"Error al aplicar filtro '{filtro_condicion}': {e}")
        return None
    if len(datos_filtrados) < 30: # (Mínimo 30 días para una regresión)
        return None 

    # 3. Preparar variables de regresión
    Y = datos_filtrados[target_analisis]
    X = datos_filtrados['Mercado']
    X = sm.add_constant(X)
    
    modelo = sm.OLS(Y, X).fit()
    
    alpha_diario = modelo.params['const']
    pvalue_alpha = modelo.pvalues['const']
    beta = modelo.params['Mercado']
    pvalue_beta = modelo.pvalues['Mercado']
    r_squared = modelo.rsquared
    
    retorno_real_anual = datos_filtrados[target_analisis].mean() * 252
    retorno_teorico_anual = rf + beta * (erm - rf)

    resultados = {
        "Hipótesis (Filtro)": filtro_condicion,
        "Target": target_analisis,
        "Días": len(datos_filtrados),
        "P Value Alpha": pvalue_alpha,
        "Alpha Estadistico Anual": alpha_diario*252,
        "Retorno esperado Anual": retorno_teorico_anual,
        "Beta": beta,
        "P Value_Beta": pvalue_beta,
        "R Squared": r_squared,
    }
    return resultados

In [6]:
def correr_analisis(datos_base, target_analisis, rf, erm):
   
    # Hipótesis 
    lista_hipotesis = [
        # Condición Base (Sin Filtro)
        "index == index", 
        
        # Cat 1: Tendencia de Mercado
        f"Precio_{ticker_mercado} > MM200",  # 1. Mercado Alcista
        f"Precio_{ticker_mercado} < MM200",  # 2. Mercado Bajista
        
        # Cat 1: Fuerza Relativa (Modificada de Cond 2 original)
        f"MA200_{target_analisis} > MM200",     # 2b. Liderazgo de Activo
        
        # Cat 2: Reversión a la Media (Activo)
        f"RSI_{target_analisis} < 30",          # 3. Sobreventa
        f"RSI_{target_analisis} > 70",          # 4. Sobrecompra
        
        # Cat 3: Volatilidad (Mercado)
        "VIX > 25",                          # 5. Pánico
        "VIX < 15",                          # 6. Calma
        
        # Cat 4: Eventos Extremos (Modificada de Cond 10 original)
        "Cond_Dias_Normales"            # 10. Días "aburridos" de mercado
    ]
    
    print(f"\n--- Corriendo análisis para: {target_analisis} ---")
    lista_de_resultados = []
    
    for hipotesis in lista_hipotesis:
        resultado = probar_condicion(
            datos_base, 
            target_analisis, 
            hipotesis, 
            rf, 
            erm
        )
        if resultado:
            lista_de_resultados.append(resultado)
    
    return pd.DataFrame(lista_de_resultados)

In [8]:
# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":

    datos_base = preparar_datos(
        tickers_cartera, weights, ticker_mercado, 
        ticker_vix, start_date, end_date
    )
    
    if datos_base is not None:
        df_resultados_cartera = correr_analisis(
            datos_base, 
            'Cartera', 
            RF_ANUAL, 
            ERM_ANUAL
        )
        
        df_resultados_activo_1 = correr_analisis(
            datos_base, 
            tickers_cartera[0], 
            RF_ANUAL, 
            ERM_ANUAL
        )
        
        print(f"\n\n--- RESULTADOS DEL ANÁLISIS: {tickers_cartera[0]} ---")
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        print(df_resultados_activo_1)
        print("\n\n--- RESULTADOS DEL ANÁLISIS: CARTERA ---")
        print(df_resultados_cartera)
       
        print("\n\n--- BÚSQUEDA DE CONDICIONES EXITOSAS (P-Value < 0.05) ---")
        
        condiciones_exitosas_A = df_resultados_activo_1 [(df_resultados_activo_1['P Value Alpha'] < 0.05) ]
        if condiciones_exitosas_A.empty:
            print("No se encontraron condiciones exitosas (P-Value < 0.05 y Convergencia) para la Cartera.")
        else:
            print("Se encontraron condiciones donde el Alpha del Activo es real:")
            print(condiciones_exitosas_A)
        
        condiciones_exitosas_C = df_resultados_cartera[(df_resultados_cartera['P Value Alpha'] < 0.05) ]    
        if condiciones_exitosas_C.empty:
            print("No se encontraron condiciones exitosas (P-Value < 0.05 y Convergencia) para la Cartera.")
        else:
            print("¡Se encontraron condiciones donde el Alpha de la Cartera es real:")
            print(condiciones_exitosas_C)

[*********************100%***********************]  5 of 5 completed

Datos preparados. 1273 días hábiles listos para analizar.

--- Corriendo análisis para: Cartera ---

--- Corriendo análisis para: GOOGL ---


--- RESULTADOS DEL ANÁLISIS: GOOGL ---
   Hipótesis (Filtro) Target  Días  P Value Alpha  Alpha Estadistico Anual  Retorno esperado Anual      Beta   P Value_Beta  R Squared
0      index == index  GOOGL  1273   4.598115e-01                 0.073867                0.092282  1.245646  1.325195e-180   0.476044
1  Precio_SPY > MM200  GOOGL  1011   9.497068e-02                 0.183999                0.094394  1.287882  2.378196e-105   0.375733
2  Precio_SPY < MM200  GOOGL   262   8.557299e-02                -0.410883                0.089671  1.193412   9.941968e-61   0.647006
3      RSI_GOOGL > 70  GOOGL   118   5.370111e-07                 2.350903                0.085944  1.118879   2.778959e-04   0.108098
4            VIX > 25  GOOGL   196   9.797228e-01                -0.006902                0.087129  1.142574   3.818398e-50   0.681948
5        

<p style="font-family:Times New Roman, serif; font-size:19px;text-align:justify">
Luego del análisis, podemos ver que tanto el activo como la cartera tuvieron 118 y 99 días en donde hubo sobrecompra con un alpha anual de 2.35 y 1.06 respectivamente, lo que concuerda con el exceso de retorno que se veía al promediar y anualizar los retornos de los 5 años en el proyecto anterior, datos estadísticos defendidos por un bajísimo p value. Además, la cartera se ve que tiende a aumentar su retorno cuando el mercado está en alza, lo cual tiene sentido ya que los 3 activos son de empresas que están en el S&P-500
</p>
<p style="font-family:Times New Roman, serif; font-size:19px;text-align:justify">
Cabe recalcar que esto se repitió para los 5 años anteriores al análisis mostrado, en donde se llega a las mismas conclusiones, lo que refuerza las observaciones.
</p>